In [1]:
import matplotlib as plt
import numpy
import json
import datetime

In [75]:
policy = 'LL'
logs1 = []
logs2 = []
logs3 = []
with open(f'YACS/worker/log_worker_1_{policy}.log') as f1, open(f'YACS/worker/log_worker_2_{policy}.log') as f2, open(f'YACS/worker/log_worker_3_{policy}.log') as f3:
    logs1 = f1.readlines()
    logs2 = f2.readlines()
    logs3 = f3.readlines()


In [76]:
logs1

['2020-12-03 22:13:11,398 %number of tasks running=% 0 %tasks completed=%0\n',
 '2020-12-03 22:13:12,415 %number of tasks running=% 0 %tasks completed=%0\n',
 '2020-12-03 22:13:13,429 %number of tasks running=% 0 %tasks completed=%0\n',
 '2020-12-03 22:13:14,444 %number of tasks running=% 0 %tasks completed=%0\n',
 '2020-12-03 22:13:15,460 %number of tasks running=% 0 %tasks completed=%0\n',
 '2020-12-03 22:13:15,720 %TASK RECEIVED%{"task_id": "1_M0", "duration": 3, "status": 1}\n',
 '2020-12-03 22:13:16,473 %number of tasks running=% 1 %tasks completed=%0\n',
 '2020-12-03 22:13:17,064 %TASK RECEIVED%{"task_id": "2_M1", "duration": 2, "status": 1}\n',
 '2020-12-03 22:13:17,374 %TASK RECEIVED%{"task_id": "3_M0", "duration": 1, "status": 1}\n',
 "2020-12-03 22:13:17,486 %TASK COMPLETED%{'task_id': '3_M0', 'duration': 0, 'status': 2}\n",
 '2020-12-03 22:13:17,498 %number of tasks running=% 2 %tasks completed=%1\n',
 '2020-12-03 22:13:17,499 %MESSAGE TO MASTER b\'{"task_id": "3_M0", "durat

In [77]:
job_received_log = []
job_completed_log = []

In [78]:
for logs in [logs1, logs2, logs3]:
    for log in logs:
        log = log.split('%')
        log[0] = log[0].strip()
        if log[1] == 'TASK RECEIVED':
            job_received_log.append(log)
        elif log[1] == 'TASK COMPLETED':
            job_completed_log.append(log)

In [79]:
job_completed_log

[['2020-12-03 22:13:17,486',
  'TASK COMPLETED',
  "{'task_id': '3_M0', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:18,509',
  'TASK COMPLETED',
  "{'task_id': '1_M0', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:18,528',
  'TASK COMPLETED',
  "{'task_id': '2_M1', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:20,553',
  'TASK COMPLETED',
  "{'task_id': '3_R0', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:20,572',
  'TASK COMPLETED',
  "{'task_id': '1_R0', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:20,573',
  'TASK COMPLETED',
  "{'task_id': '1_R1', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:20,596',
  'TASK COMPLETED',
  "{'task_id': '5_M3', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:24,667',
  'TASK COMPLETED',
  "{'task_id': '4_R0', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:24,674',
  'TASK COMPLETED',
  "{'task_id': '4_R1', 'duration': 0, 'status': 2}\n"],
 ['2020-12-03 22:13:24,693',
  'TASK COMPLETED',
  "{'t

In [80]:
for i in job_received_log:
    i[2] = json.loads(i[2])['task_id']    
for i in job_completed_log:
    #i[2] = json.loads(i[2])['task_id']    
    i[2] = i[2].split(',')[0].split(':')[1].strip()  

In [81]:
print(job_received_log[1])
print(job_completed_log[1])

['2020-12-03 22:13:17,064', 'TASK RECEIVED', '2_M1']
['2020-12-03 22:13:18,509', 'TASK COMPLETED', "'1_M0'"]


In [82]:
def convert_to_datetime(datetime_string):
    k = datetime.datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S,%f')
    return k

In [83]:
job_times = {}
for i in job_received_log:
    for j in job_completed_log:
        if i[2] == j[2][1:-1]:
            job_times[i[2]] = (convert_to_datetime(j[0]) - convert_to_datetime(i[0])).total_seconds()

In [84]:
len(job_times)

406

In [85]:
avg_times[policy] = sum(list(job_times.values()))/len(job_times)
sorted_times = sorted(list(job_times.values()))
median_times[policy] = (sorted_times[49] + sorted_times[50])/2

In [86]:
avg_times

{'RR': 2.0691945812807875, 'RD': 2.0827733990147768, 'LL': 2.1080689655172398}

In [87]:
median_times

{'RR': 0.522, 'RD': 0.577, 'LL': 0.6315}